In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import time

import warnings
warnings.filterwarnings('ignore')

start = datetime.now()

In [2]:
df = pd.read_csv(r'D:\SATYAM\UC Analyst\Copy of Data Engineer Assignment.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945726 entries, 0 to 1945725
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   provider_id  object
 1   detail       object
 2   source       object
 3   event_time   object
dtypes: object(4)
memory usage: 59.4+ MB


In [3]:
# Spilitting date and time
df2 = df['event_time'].str.split(expand=True)   
df2.columns = ['Date', 'Time']
df2['Time'] = df2["Time"].str.replace(".000000","",regex=True)

In [4]:
# A dataframe with seperate time and date columns
df3 = pd.concat([df,df2],axis=1)


In [5]:
#Values between 7PM and midnight
df4 = df3.copy()



In [6]:
# Replacing the values between midnight and 8AM as online at 8AM
df4.loc[df4["Time"].between('00:00:00', '08:00:00', inclusive=False), "Time"] = '08:00:00'
df4.loc[df4["Time"].between('19:00:00', '23:59:59', inclusive=True), "Time"] = '18:59:59'

In [7]:
# Getting values where detail is Flase
df_false = df4[df4['detail'].str.contains('False',regex = False, na = False)]


In [8]:
#Getting values where detail is true and source is action on job
df_true = df4[(df4['detail'].str.contains('True',regex = False) | df4['source'].str.contains('Action on Job',regex = False))]

In [9]:
#Getting values where detail is false and source is action on job / Both online and offline
df_both = df4[ (df4['source'].str.contains('Action on Job',regex = False)) & (df4['detail'].str.contains('False',regex = False, na = False)) ]  

In [10]:
#Designating online and offline status
df_false['Status'] = 'Offline'
df_true['Status'] = 'Online'
df_both['Status'] = 'Online'

In [11]:
df5 = pd.concat([df_true, df_false,df_both],axis = 0,ignore_index=True).drop_duplicates(keep=False)
df5

,provider_id,detail,source,event_time,Date,Time,Status
0,4f3453ed4dd693aebe3532d0f3da717f,{u'mme': True},manual,2017-09-01 00:01:00.000000,2017-09-01,08:00:00,Online
1,72da31acf3cb317a640309de031dba1d,{u'mme': True},manual,2017-09-01 00:01:07.000000,2017-09-01,08:00:00,Online
2,421fb73595f2646bce5fba6ab33fe1df,{u'mme': True},manual,2017-09-01 00:08:13.000000,2017-09-01,08:00:00,Online
3,72da31acf3cb317a640309de031dba1d,{u'mme': True},manual,2017-09-01 00:13:45.000000,2017-09-01,08:00:00,Online
4,72da31acf3cb317a640309de031dba1d,{u'paf': True},manual,2017-09-01 00:14:40.000000,2017-09-01,08:00:00,Online
...,...,...,...,...,...,...,...
888118,c08c00053a6be225200c891810df9d4f,{u'mme': False},decline_request,2017-09-30 23:58:38.000000,2017-09-30,18:59:59,Offline
888119,cfdd7d7604c24c72d732a576e438a362,{u'mme': False},decline_request,2017-09-30 23:58:48.000000,2017-09-30,18:59:59,Offline
888120,9094a7169ad69befb34ed57640e12132,{u'paf': False},manual,2017-09-30 23:58:50.000000,2017-09-30,18:59:59,Offline
888121,cb817c584975076f22d4bd7d4b21b29e,{u'mme': False},decline_request,2017-09-30 23:59:00.000000,2017-09-30,18:59:59,Offline


In [12]:
df5.sort_values('provider_id',ascending = True,ignore_index = True,inplace = True)


In [13]:
df5.drop(columns= ['detail','source','event_time'],inplace = True)


In [14]:
sf = df5.copy().drop_duplicates()
sf['dt']= df5['Date'].astype(str) + ' ' +  df5['Time'].astype(str)
sf['dt'] = pd.to_datetime(sf['dt'])

In [42]:
sf0 = sf[['provider_id','dt','Status','Date','Time']].drop_duplicates()

x = sf0.loc[(sf0['provider_id' ] == '0037dfffff8b03bbdf366a263735e84b') & (sf0['Date'] == '2017-09-05')]
x.sort_values('dt')

,provider_id,dt,Status,Date,Time
515,0037dfffff8b03bbdf366a263735e84b,2017-09-05 09:00:13,Online,2017-09-05,09:00:13
752,0037dfffff8b03bbdf366a263735e84b,2017-09-05 09:29:21,Offline,2017-09-05,09:29:21
773,0037dfffff8b03bbdf366a263735e84b,2017-09-05 09:29:43,Offline,2017-09-05,09:29:43
681,0037dfffff8b03bbdf366a263735e84b,2017-09-05 11:14:30,Offline,2017-09-05,11:14:30
794,0037dfffff8b03bbdf366a263735e84b,2017-09-05 11:14:30,Online,2017-09-05,11:14:30
454,0037dfffff8b03bbdf366a263735e84b,2017-09-05 13:15:55,Online,2017-09-05,13:15:55
446,0037dfffff8b03bbdf366a263735e84b,2017-09-05 15:29:36,Offline,2017-09-05,15:29:36
444,0037dfffff8b03bbdf366a263735e84b,2017-09-05 15:30:26,Offline,2017-09-05,15:30:26
621,0037dfffff8b03bbdf366a263735e84b,2017-09-05 17:11:18,Online,2017-09-05,17:11:18
761,0037dfffff8b03bbdf366a263735e84b,2017-09-05 17:11:18,Offline,2017-09-05,17:11:18


In [16]:
import calendar
num_days = calendar.monthrange(2017, 9)[1]
days = pd.date_range('2017-09-01','2017-09-30').tolist()
date1 = pd.DataFrame({'Date':days})    

sf5 = sf0[['provider_id']].drop_duplicates().merge(date1, how='cross')


tf = pd.date_range( '08:00:00','19:00:00',freq='H').time
time1 = pd.DataFrame({'Time':tf})    

sf5 = sf5.merge(time1, how='cross')
sf5['dt'] = sf5['Date'].astype(str) + ' ' + sf5['Time'].astype(str)
#sf5['Date'] = sf5['dt'].dt.date
sf5['Date'] = sf5['Date'].dt.date

In [17]:
sf5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5861880 entries, 0 to 5861879
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   provider_id  object
 1   Date         object
 2   Time         object
 3   dt           object
dtypes: object(4)
memory usage: 223.6+ MB


In [18]:
sf5['Status'] = np.nan
sf5['Time'] = sf5['Time'].astype(str)

In [19]:
sf6 = pd.concat([sf0,sf5])
sf6
sf6['Date'] = sf6['Date'].astype(str)

sf6.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6501311 entries, 0 to 5861879
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   provider_id  object
 1   dt           object
 2   Status       object
 3   Date         object
 4   Time         object
dtypes: object(5)
memory usage: 297.6+ MB


In [20]:
sf7 = sf6.sort_values(['provider_id','Date','Time']).reset_index(drop = True)

tf2 = sf7.ffill().drop_duplicates()

tf2.loc[(tf2.Time == tf2.Time.shift()), "Status" ] = "Online" 

tf2 = tf2.drop_duplicates()

tf2.loc[(tf2['Time'] == '08:00:00') & ((tf2['Time'] < tf2['Time'].shift()) | (tf2.Time.shift().isna())), 'Status' ] = 'Offline' 




tf2.drop_duplicates(inplace = True)
tf2 = tf2.ffill()

In [21]:
tf2

,provider_id,dt,Status,Date,Time
0,00020da44b0a44e867de5f26c69886ac,2017-09-01 08:00:00,Offline,2017-09-01,08:00:00
1,00020da44b0a44e867de5f26c69886ac,2017-09-01 09:00:00,Offline,2017-09-01,09:00:00
2,00020da44b0a44e867de5f26c69886ac,2017-09-01 10:00:00,Offline,2017-09-01,10:00:00
3,00020da44b0a44e867de5f26c69886ac,2017-09-01 11:00:00,Offline,2017-09-01,11:00:00
4,00020da44b0a44e867de5f26c69886ac,2017-09-01 12:00:00,Offline,2017-09-01,12:00:00
...,...,...,...,...,...
6501306,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30 15:00:00,Offline,2017-09-30,15:00:00
6501307,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30 16:00:00,Offline,2017-09-30,16:00:00
6501308,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30 17:00:00,Offline,2017-09-30,17:00:00
6501309,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30 18:00:00,Offline,2017-09-30,18:00:00


In [22]:
tf2.Time = pd.to_datetime(tf2.Time)

In [23]:
tf2['Online_time'] = ((tf2.Time).shift(-1) - (tf2.Time)).dt.total_seconds().fillna(0)
tf2.loc[(tf2.Status == 'Offline') | (tf2.Time.astype(str) == '19:00:00'), 'Online_time' ] = 0

In [24]:
# tf2.loc[(tf2['provider_id' ] == '0037dfffff8b03bbdf366a263735e84b') & (tf2['Date'].astype(str) == '2017-09-04')]

In [25]:
res = tf2.groupby(['provider_id','Date', pd.Grouper(freq='H',key = "Time")]).sum()

res.reset_index(inplace = True)

In [26]:
res = res[~(res['Time'].astype(str).str.contains('19:00:00',regex = True))]

res['Hour_end'] = res['Time'] + pd.DateOffset(hours = 1)

res.Time = res.Time.dt.time

res.Hour_end = res.Hour_end.dt.time



In [27]:
res.loc[(res['provider_id' ] == '0037dfffff8b03bbdf366a263735e84b') & (res['Date'].astype(str) == '2017-09-05')]

,provider_id,Date,Time,Online_time,Hour_end
6888,0037dfffff8b03bbdf366a263735e84b,2017-09-05,08:00:00,0.0,09:00:00
6889,0037dfffff8b03bbdf366a263735e84b,2017-09-05,09:00:00,1761.0,10:00:00
6890,0037dfffff8b03bbdf366a263735e84b,2017-09-05,10:00:00,0.0,11:00:00
6891,0037dfffff8b03bbdf366a263735e84b,2017-09-05,11:00:00,2730.0,12:00:00
6892,0037dfffff8b03bbdf366a263735e84b,2017-09-05,12:00:00,3600.0,13:00:00
6893,0037dfffff8b03bbdf366a263735e84b,2017-09-05,13:00:00,3600.0,14:00:00
6894,0037dfffff8b03bbdf366a263735e84b,2017-09-05,14:00:00,3600.0,15:00:00
6895,0037dfffff8b03bbdf366a263735e84b,2017-09-05,15:00:00,1776.0,16:00:00
6896,0037dfffff8b03bbdf366a263735e84b,2017-09-05,16:00:00,0.0,17:00:00
6897,0037dfffff8b03bbdf366a263735e84b,2017-09-05,17:00:00,2922.0,18:00:00


In [28]:
sf0.loc[(sf0['provider_id' ] == '0037dfffff8b03bbdf366a263735e84b') & (sf0['Date'].astype(str) == '2017-09-05')].sort_values('Time')

,provider_id,dt,Status,Date,Time
515,0037dfffff8b03bbdf366a263735e84b,2017-09-05 09:00:13,Online,2017-09-05,09:00:13
752,0037dfffff8b03bbdf366a263735e84b,2017-09-05 09:29:21,Offline,2017-09-05,09:29:21
773,0037dfffff8b03bbdf366a263735e84b,2017-09-05 09:29:43,Offline,2017-09-05,09:29:43
681,0037dfffff8b03bbdf366a263735e84b,2017-09-05 11:14:30,Offline,2017-09-05,11:14:30
794,0037dfffff8b03bbdf366a263735e84b,2017-09-05 11:14:30,Online,2017-09-05,11:14:30
454,0037dfffff8b03bbdf366a263735e84b,2017-09-05 13:15:55,Online,2017-09-05,13:15:55
446,0037dfffff8b03bbdf366a263735e84b,2017-09-05 15:29:36,Offline,2017-09-05,15:29:36
444,0037dfffff8b03bbdf366a263735e84b,2017-09-05 15:30:26,Offline,2017-09-05,15:30:26
621,0037dfffff8b03bbdf366a263735e84b,2017-09-05 17:11:18,Online,2017-09-05,17:11:18
761,0037dfffff8b03bbdf366a263735e84b,2017-09-05 17:11:18,Offline,2017-09-05,17:11:18


In [29]:
res.rename(columns = {'Time':'Hour_start'}, inplace = True)

In [30]:
res.Online_time = res.Online_time.astype(int)

In [31]:
res.reset_index(drop = True,inplace = True)
result = res[~(res['Hour_start'].astype(str) == '00:00:00')] 

In [32]:
result

,provider_id,Date,Hour_start,Online_time,Hour_end
0,00020da44b0a44e867de5f26c69886ac,2017-09-01,08:00:00,0,09:00:00
1,00020da44b0a44e867de5f26c69886ac,2017-09-01,09:00:00,0,10:00:00
2,00020da44b0a44e867de5f26c69886ac,2017-09-01,10:00:00,0,11:00:00
3,00020da44b0a44e867de5f26c69886ac,2017-09-01,11:00:00,0,12:00:00
4,00020da44b0a44e867de5f26c69886ac,2017-09-01,12:00:00,0,13:00:00
...,...,...,...,...,...
5373391,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30,14:00:00,0,15:00:00
5373392,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30,15:00:00,0,16:00:00
5373393,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30,16:00:00,0,17:00:00
5373394,fff9bad31994b4dd3d5859dac8c8ed7e,2017-09-30,17:00:00,0,18:00:00


In [33]:
end = datetime.now()
end - start

datetime.timedelta(seconds=149, microseconds=810779)

In [38]:
result.loc[(result['provider_id' ] == '0037dfffff8b03bbdf366a263735e84b') & (result['Date'].astype(str) == '2017-09-05')].sort_values('Hour_start')

,provider_id,Date,Hour_start,Online_time,Hour_end
6314,0037dfffff8b03bbdf366a263735e84b,2017-09-05,08:00:00,0,09:00:00
6315,0037dfffff8b03bbdf366a263735e84b,2017-09-05,09:00:00,1761,10:00:00
6316,0037dfffff8b03bbdf366a263735e84b,2017-09-05,10:00:00,0,11:00:00
6317,0037dfffff8b03bbdf366a263735e84b,2017-09-05,11:00:00,2730,12:00:00
6318,0037dfffff8b03bbdf366a263735e84b,2017-09-05,12:00:00,3600,13:00:00
6319,0037dfffff8b03bbdf366a263735e84b,2017-09-05,13:00:00,3600,14:00:00
6320,0037dfffff8b03bbdf366a263735e84b,2017-09-05,14:00:00,3600,15:00:00
6321,0037dfffff8b03bbdf366a263735e84b,2017-09-05,15:00:00,1776,16:00:00
6322,0037dfffff8b03bbdf366a263735e84b,2017-09-05,16:00:00,0,17:00:00
6323,0037dfffff8b03bbdf366a263735e84b,2017-09-05,17:00:00,2922,18:00:00


In [39]:
result.shape

(5373390, 5)

In [40]:
result.to_csv('provider_data_detail.csv')